In [ ]:
pip install tika

In [51]:
# 의사록 Crawling

import requests
from bs4 import BeautifulSoup
from tika import parser
import pandas as pd

start_page = 1
current_page = start_page
end_page = 31

# 32페이지는 링크따와서 수작업
host = 'http://www.bok.or.kr/'
result_df = pd.DataFrame(columns=['date', 'text'])

while True:

    URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(current_page)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text, 'html.parser')
    results = sp.select('div.fileGoupBox ul')
    
    for i, result in enumerate(results):
        pdf_date = sp.select('span.date')[i].text[3:]
        
        pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

        with open('bok.pdf', 'wb') as f:
            f.write(pdf_temp.content)
        
        parsed = parser.from_file('bok.pdf')
        temp_text = str(parsed['content']).strip()
        temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
        result_df = pd.concat([result_df, temp_df])

    if current_page == end_page:
        result_df.to_csv('datas.csv')
        break
    current_page += 1


In [113]:
# 32 페이지 수작업 부분
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

for i, result in enumerate(results):
    pdf_date = sp.select('span.date')[i].text[3:]

    pdf_temp = requests.get(host + result.select('li')[1].select('a')[0].attrs['href'])

    with open('bok.pdf', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.pdf')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    test_df = pd.concat([test_df, temp_df])

In [111]:
URL = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=32'
r = requests.get(URL)
sp = BeautifulSoup(r.text, 'html.parser')
results = sp.select('div.fileGoupBox ul')

In [78]:
results = sp.select('div.fileLink')

In [88]:
results[1].select('a')[0].attrs['href']

'/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000085858&fileSn=1'

In [74]:
pdf_date = sp.select('span.date')[7].text[3:]

In [101]:
for i, result in enumerate(results):
    if i==0:
        continue
    
    pdf_date = sp.select('span.date')[i].text[3:]

    pdf_temp = requests.get(host + results[i].select('a')[0].attrs['href'])

    with open('bok.hwp', 'wb') as f:
        f.write(pdf_temp.content)

    parsed = parser.from_file('bok.hwp')
    temp_text = str(parsed['content']).strip()
    temp_df = pd.DataFrame([{'date':pdf_date, 'text':temp_text}])
    test_df = pd.concat([test_df, temp_df])



In [106]:
test_df.to_csv('bok_data.csv',encoding='utf-8')

In [125]:
# pd.read_csv('bok_data.csv').iloc[309:]

In [116]:
test_df.to_csv('bok_data.csv',encoding='utf-8')

In [124]:
# pd.read_csv('bok_data.csv').iloc[319]['text']